In [57]:
import gensim
LabeledSentence=gensim.models.doc2vec.LabeledSentence
TaggedDocument= gensim.models.doc2vec.TaggedDocument

In [3]:
from sklearn.cross_validation import train_test_split
import numpy as np

In [14]:
import pandas as pd
import string

In [264]:
train_data=pd.read_csv(r'C:\Users\chaoqun chen\Desktop\kaggle比赛\train.csv')
test_data=pd.read_csv(r'C:\Users\chaoqun chen\Desktop\kaggle比赛\test.csv')

In [265]:
x_train,x_val,y_train,y_val=train_test_split(train_data['text'],train_data['author'],\
                                            test_size=0.3,random_state=2017)

In [266]:
x_train_id = train_data[train_data.index.isin(x_train.index)]['id']

In [267]:
x_val_id = train_data[train_data.index.isin(x_val.index)]['id']

In [268]:
def cleanText(corpus):
    punctuation = """.,?!:;(){}[]"""
    corpus = [z.lower().replace('\n','') for z in corpus]
    #将标点符号也当做字符
    for c in punctuation:
        corpus = [z.replace(c, " %s"%c) for z in corpus]
    corpus = [z.split() for z in corpus]
    return corpus

In [269]:
x_train = cleanText(x_train)
x_val = cleanText(x_val)
test_texts = cleanText(test_data['text'])

In [270]:
#gensim的word2vec需要每个文档都有一个label
def LabeledLineSentence(texts, id_list, label_type):
    for text,id in zip(texts,id_list):
        label = '%s_%s'%(label_type, id)
        yield LabeledSentence(text,[label])

In [290]:
Tagged =[]

In [291]:
def TaggedText_allCorpus(texts, Tagged,id_list, label_type):
    #Tagged =[]
    for text,id in zip(texts,id_list):
        label = '%s_%s'%(label_type, id)
        Tagged.append(TaggedDocument(text,[label]))
    #return Tagged

In [292]:
TaggedText_allCorpus(x_train,Tagged,x_train_id,'TRAIN')
print(len(Tagged))
TaggedText_allCorpus(x_val,Tagged,x_val_id,'VAL')
TaggedText_allCorpus(test_texts,Tagged,test_data['id'],'TEST')

13705


In [293]:
all_corpus=Tagged

In [294]:
len(all_corpus)

27971

In [260]:
def TaggedText(texts, id_list, label_type):
    Tagged =[]
    for text,id in zip(texts,id_list):
        label = '%s_%s'%(label_type, id)
        Tagged.append(TaggedDocument(text,[label]))
    return Tagged

In [295]:
x_train=TaggedText(x_train,x_train_id,'TRAIN')
#x_train=LabeledLineSentence(x_train,x_train_id,'TRAIN')

In [297]:
print(type(x_train),len(x_train))

<class 'list'> 13705


In [245]:
x_train[0:2]

[TaggedDocument(words=['i', 'took', 'a', 'frenzied', 'pleasure', 'in', 'contrasting', 'its', 'downward', 'with', 'its', 'lateral', 'velocity', '.'], tags=['TEST_id02759']),
 TaggedDocument(words=['an', 'inquiry', 'will', 'afford', 'us', 'amusement', ',"', 'i', 'thought', 'this', 'an', 'odd', 'term', ',', 'so', 'applied', ',', 'but', 'said', 'nothing', '"and', ',', 'besides', ',', 'le', 'bon', 'once', 'rendered', 'me', 'a', 'service', 'for', 'which', 'i', 'am', 'not', 'ungrateful', '.'], tags=['TRAIN_id08378'])]

In [307]:
len(x_train)

13705

In [298]:
x_val=TaggedText(x_val,x_val_id,'VAL')

In [299]:
x_test = TaggedText(test_texts,test_data['id'],'TEST')

Doc2Vec 的两个模型，DM 和 DBOW。
gensim 的说明文档建议多次训练数据集并调整学习速率或在每次训练中打乱输入信息的顺序。
我们从Doc2Vec 模型中获得向量。

In [179]:
import random
size=400

In [300]:
#min_count = ignore all words with total frequency lower than this.
model_dm=gensim.models.doc2vec.Doc2Vec(min_count=1,window=10,size=size,sample=1e-3,negative=5,
                              workers=3)

In [301]:
model_dbow=gensim.models.doc2vec.Doc2Vec(min_count=1,window=10,size=size,sample=1e-3,negative=5,
                              dm=0,workers=3)

In [184]:
#all_corpus=list(np.concatenate((x_train,x_val,x_test)))

#all_corpus=x_train+x_val+x_test

#all_corpus[0:2]

In [302]:
model_dm.build_vocab(all_corpus)
model_dbow.build_vocab(all_corpus)

In [303]:
model_dm

In [186]:
#model_dm.build_vocab(x_train)
#model_dbow.build_vocab(x_train)

In [230]:
np.random.shuffle(all_corpus)

In [231]:
all_corpus[0:2]

[TaggedDocument(words=['"what', 'do', 'you', 'mean', 'by', 'interfering', 'with', 'a', 'gentleman', 'in', 'his', 'pleasures', '?"'], tags=['TEST_id10286']),
 TaggedDocument(words=['even', 'perdita', 'will', 'rejoice', '.'], tags=['VAL_id25833'])]

In [304]:
for epoch in range(10):
    #perm = np.random.permutation(len(all_corpus))
    #print(list(map(int,perm)))
    #perm=list(map(int,perm))
    np.random.shuffle(all_corpus)
    model_dm.train(all_corpus,total_examples=model_dm.corpus_count,\
                   epochs=model_dm.iter)
    model_dbow.train(all_corpus,total_examples=model_dbow.corpus_count,\
                    epochs=model_dbow.iter)

In [308]:
model_dm.save(r'C:\Users\chaoqun chen\Desktop\kaggle比赛\model\doc2vec_dm.doc2vec')

In [309]:
model_dbow.save(r'C:\Users\chaoqun chen\Desktop\kaggle比赛\model\doc2vec_dbow.doc2vec')

In [314]:
def getVecs(model,corpus,size):
    Vecs=[np.array(model.infer_vector(z.words)).reshape((1,size)) for z in corpus]
    return np.concatenate(Vecs)

In [315]:
train_vecs_dm = getVecs(model_dm,x_train,size)
train_vecs_dbow = getVecs(model_dbow,x_train,size)

In [316]:
 train_vecs_dm[0:2]

array([[  1.18000388e-01,   3.10263429e-02,   8.90923887e-02,
          4.79938835e-02,   1.51903912e-01,   1.12465613e-01,
         -1.48219392e-01,  -1.99942384e-02,   1.63476467e-01,
         -5.47001511e-02,   1.65423945e-01,   8.93721059e-02,
          7.17518255e-02,  -2.72216320e-01,   1.03495065e-02,
          1.81832649e-02,   4.97806594e-02,  -1.94909975e-01,
          2.59723574e-01,  -1.89567097e-02,   2.39867102e-02,
          2.60039210e-01,  -3.34590836e-03,  -8.00897777e-02,
          9.18378755e-02,   4.35853451e-02,   1.52359992e-01,
          1.66566268e-01,  -1.63739535e-03,   3.97743694e-02,
         -3.77949178e-02,  -3.94336469e-02,  -1.17513873e-01,
          9.19104517e-02,  -6.43970817e-02,  -2.03214325e-02,
          2.58856395e-04,   1.33060649e-01,   5.89993559e-02,
          1.23848170e-02,  -1.34755492e-01,   2.26454645e-01,
          6.59032241e-02,  -1.04687989e-01,   1.18967697e-01,
         -1.33046033e-02,  -1.65667146e-01,   1.53644294e-01,
        

In [317]:
from sklearn.linear_model import SGDClassifier

In [331]:
sgd = SGDClassifier(loss='log',penalty='l2')
sgd.fit(train_vecs_dm,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [332]:
val_vecs_dm = getVecs(model_dm,x_val,size)

In [333]:
print(sgd.score(val_vecs_dm,y_val))

0.545114061968


In [335]:
from sklearn.metrics import log_loss

0.957885240643


In [336]:
def get_logloss(model,x_train,y_train,x_val,y_val,metrics):
    model.fit(x_train,y_train)
    y_pred_proba=model.predict_proba(x_val)
    print(metrics(y_val,y_pred_proba))

In [337]:
get_logloss(sgd,train_vecs_dm,y_train,val_vecs_dm,y_val,log_loss)

0.958131000199


In [338]:
from sklearn.ensemble import RandomForestClassifier
rfc= RandomForestClassifier()
get_logloss(rfc,train_vecs_dm,y_train,val_vecs_dm,y_val,log_loss)

1.77363194582


In [339]:
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression()
get_logloss(lr,train_vecs_dm,y_train,val_vecs_dm,y_val,log_loss)

0.926044323988
